Import variables, functions, and construct file structures

In [ ]:
from resource_manager import get_data_from_source
from environment import output_path
from functions import separate_vocal, convert_ncm, apply_so_vits, fuse_vocal_and_instrumental, Path

Define paths

In [ ]:
SONG_PATH = Path("./demo_assets/minstrel_short.mp3")
OUT_PATH = output_path.joinpath(SONG_PATH.name.rsplit('.')[0])

Download the model from huggingface

In [ ]:
model_so_vits_genshin = get_data_from_source("so-vits", "model", "genshin", update_cache=False)
model_so_vits_hololive = get_data_from_source("so-vits", "model", "hololive", update_cache=False)
model_demucs = get_data_from_source("demucs", "model", "hdemucs_mmi", update_cache=False)

Convert .ncm file to .wav

In [ ]:
converted_path = convert_ncm(SONG_PATH, OUT_PATH)

Separate vocal and instrumental with demucs

In [ ]:
separated_path = separate_vocal(Path(converted_path), OUT_PATH)
print(separated_path)

Use so-vits-svc to process audio file

In [ ]:

counterfeited_path = apply_so_vits(separated_path["vocal"], output_path=OUT_PATH, model_path=model_so_vits_genshin["hutao-jp/hutao_jp_G_40000.pth"], cluster=model_so_vits_genshin["hutao-jp/hutao_jp_kmeans_10000.pt"], config_file_path=model_so_vits_genshin["hutao-jp/hutao.json"], auto_predict_f0=False, speaker="hutao", db_threshold=0, chunk_seconds=40)


print(counterfeited_path)

## 合并

In [ ]:
output = fuse_vocal_and_instrumental(vocal_path=counterfeited_path, instrumental_path=separated_path["instrumental"], output_path=OUT_PATH, speaker="hutao")
print("output file:", output)